# Clean the fasta file

In [1]:
import numpy as np

In [24]:
#read fasta file and get sequences
# For some reason, sequences_full is a bit fucked up, with non-canonical bases and same names
fasta_file = 'data/output_human_mrna.fa'

sequences = {}

with open(fasta_file, 'r') as file:
    lines = file.readlines()
    for i in range(0, len(lines), 2):
        seq_id = lines[i][1:].strip()

        if seq_id in sequences:
            k = 1
            while seq_id in sequences: 
                seq_id = seq_id.split('_')[0] + f'_{k}'
                k +=1
        
        sequences[seq_id] = lines[i+1].strip().upper().replace('T', 'U')

names = [key for key in sequences.keys()]
sequences = [item for item in sequences.values()]
print(len(sequences))

mask_valid = np.array([True if set([*seq]) == set(['A', 'U', 'C', 'G']) else False for seq in sequences])

names = np.array(names)[mask_valid]
sequences = np.array(sequences)[mask_valid]

print(len(sequences))

119916
119916


In [27]:
import plotly.express as px
print(min([len(seq) for seq in sequences])
, max([len(seq) for seq in sequences]))
px.histogram([len(seq) for seq in sequences])

78 1024


In [29]:
print('Saving sequences')
with open(fasta_file, 'w') as f:
    for name, seq in zip(names, sequences):
        f.write('>'+name+'\n')
        f.write(seq+'\n')

Saving sequences


# Now the data is clean, let's import it to rouskinhf

In [4]:
import envbash
envbash.load.load_envbash('../.env')
from rouskinhf import convert, upload_dataset, get_dataset

In [31]:
import pandas as pd

data = convert(
    'json',
    f"data/output_human_mrna.fa.json",
    name='mRNA_synthetic',
    path_out='data',
    filter=True,   
)
print(pd.DataFrame(data).T)


Parsing json file: 100%|██████████| 119542/119542 [00:01<00:00, 67927.89it/s]


Over a total of 119542 datapoints, there are:
### OUTPUT
- ALL: 119141 valid datapoints
- INCLUDED: 0 duplicate sequences with different structure / dms / shape
### MODIFIED
- 0 multiple sequences with the same reference (renamed reference)
### FILTERED OUT
- 0 invalid datapoints (ex: sequence with non-regular characters)
- 0 datapoints with bad structures
- 401 duplicate sequences with the same structure / dms / shape
                                                                   sequence  \
CCDS2.2|Hs110|chr1        AUGUCCAAGGGGAUCCUGCAGGUGCAUCCUCCGAUCUGCGACUGCC...   
CCDS2.2|Hs110|chr1_1      GCCUGGCAGCAGGAGCUCCUGCGGAAGCAGAACCUGGCCCGGCUGG...   
CCDS3.1|Hs110|chr1        AUGGCAGCUGCGGGGAGCCGCAAGAGGCGCCUGGCGGAGCUGACGG...   
CCDS3.1|Hs110|chr1_1      AGGAAAGUGCUGAGGCCAACAAAUUCCAGGUCACGGACAGUGCUGC...   
CCDS3.1|Hs110|chr1_2      CACGCCUUCCUCUACAUCCGCCAGCUCGCCAUACACCUGCGCAACG...   
...                                                                     ...   
CCDS94720.1|Hs110|chrY  

## To HuggingFace

In [32]:
upload_dataset(
    'data/mRNA_synthetic/data.json',
    commit_message='first commit',
    exist_ok=True,
)

data.json:   0%|          | 0.00/284M [00:00<?, ?B/s]